In [11]:
import os
import fnmatch
import cv2
import numpy as np
import string
import time
import json

from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior
#ignore warnings in the output
tf.logging.set_verbosity(tf.logging.ERROR)

In [13]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2075517531822765546
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3149044121
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14525844222967864360
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:3a:00.0, compute capability: 6.1"
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050, pci bus id: 0000:3a:00.0, compute capability: 6.1



In [19]:
 with open('./label_word_num.json',encoding='UTF8') as f:
    label_word_num = json.load(f)
with open('./label_num_word.json',encoding='UTF8') as f:
    label_num_word = json.load(f)
with open('./label_text.json',encoding='UTF8') as f:
    label_text = json.load(f)
with open('./img_list.json',encoding='UTF8') as f:
    img_list = json.load(f)

In [15]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(label_num_word)+1, activation = 'softmax')(blstm_2)
 
act_model = Model(inputs, outputs)

In [16]:
act_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 128, 1)]      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 128, 64)       640       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 64, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 8, 32, 128)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 32, 256)        295168    
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 32, 256)        590080

In [17]:
labels = Input(name='the_labels', shape=[17], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [18]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
 
filepath="best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [22]:
from keras.models import Sequential
from data_generator import DataGenerator

train_data = DataGenerator(img_list, label_text, 2)
# train, val, test 나눠서 해볼것

In [23]:
train_data